<a href="https://colab.research.google.com/github/s1scottd/CIFAR-10_Image-Classification-CNN/blob/main/CIFAR_10_Image_Classification_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Import Libraries. 

In [ ]:
import os
import tensorflow as tf
import matplotlib as mplt
import matplotlib.pyplot as plt
import matplotlib.gridspec as gs
import numpy as np

from tensorflow import keras
from keras.datasets import mnist
from keras import layers
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

from keras.datasets import cifar10
print(f"tensorflow version: {tf.__version__}")
print(f"numpy version: {np.__version__}")
print(f"keras version: {keras.__version__}")

##Prepare the data

Define a reshape and normalize function

In [ ]:
def normalize(data):
  data = data/255.
  return data

Load, Reshape and Normalize the CIFR-10 Dataset

In [ ]:
# Load CIFAR-10 dataset
(X_train, y_train), (X_test, y_test) = cifar10.load_data()

X_train = X_train[:-10000]
X_val = X_train[-10000:]
y_train = y_train[:-10000]
y_val = y_train[-10000:]

X_train = normalize(X_train)
X_test = normalize(X_test)
X_val = normalize(X_val)
print(f"X_train shape: {X_train.shape}")
print(f"X_test shape:  {X_test.shape}")
print(f"X_val shape: {X_val.shape}")

Define an image display function

In [ ]:


def display_images(data, labels):
  
  labels = labels.reshape(-1)

  labels_dict = {
    0: 'airplane',
    1: 'automobile',
    2: 'bird',
    3: 'cat',
    4: 'deer',
    5: 'dog',
    6: 'frog',
    7: 'horse',
    8: 'ship',
    9: 'truck'
  }

  fig, axes = plt.subplots(3, 3, figsize=(10, 10))

  for i, ax in enumerate(axes.flat):
    img = data[i]
    label = labels[i]
    
    ax.imshow(img)
    ax.set_title(f"{labels_dict[label]}")
    ax.axis('off')

  plt.show()

Here are the first 9 images

In [ ]:
# visualize data by plotting images
display_images(X_train, y_train)

# Convolutional Neural Network without Tuning

Define and compile the CNN model

In [ ]:
# Define the CNN model
model = Sequential([
    Conv2D(64, (3, 3), activation='relu', input_shape=(32, 32, 3)),
    MaxPooling2D((2, 2)),
    Dropout(0.25),

    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Dropout(0.25),

    Flatten(),
    Dense(64, activation='relu'),
    Dropout(0.5),
    Dense(10)
])

# Compile the model
model.compile(optimizer='adam', 
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), 
              metrics=['accuracy'])

model.summary()

Create a callback that will stop fitting the model once it reaches 95% accuracy.

In [ ]:
class callback_98(tf.keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs=[]):
    if (logs.get("val_accuracy") >= 0.98):
      self.model.stop_training = True
      print(f"\nReached 98% accuracy so cancelling training after {epoch} epochs.\n")

Fit the CNN model

In [ ]:
# Train the model
callbacks = callback_98()
history = model.fit(X_train, y_train, epochs=100, batch_size=64, validation_data=(X_val, y_val), callbacks=[callbacks])

Loss and Accuracy

In [ ]:
plt.subplot(2, 1, 1)
plt.title('Loss')
plt.plot(history.history['loss'], color='blue', label='train')
plt.plot(history.history['val_loss'], color='orange', label='test')
plt.legend(loc="upper right")

plt.subplot(2, 1, 2)
plt.title('Accuracy')
plt.plot(history.history['accuracy'], color='blue', label='train')
plt.plot(history.history['val_accuracy'], color='orange', label='test')
plt.legend(loc="lower right")

plt.subplots_adjust(hspace=0.5)

Evaluate the model

In [ ]:
# Evaluate the model
test_loss, test_acc = model.evaluate(X_test,  y_test, verbose=2)

# Convolutional Neural Network with Tuning

Install and import Tuner

In [ ]:
!pip install keras-tuner --upgrade
import keras_tuner
keras_tuner_version = keras_tuner.__version__
print(f"keras_tuner version: {keras_tuner_version}")

Create a tunable model

In [ ]:


def build_model(hp):
  model = Sequential()

  model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3)))
  model.add(MaxPooling2D((2, 2)))
  model.add(Conv2D(64, (3, 3), activation='relu'))
  model.add(MaxPooling2D((2, 2)))
  
  model.add(Flatten())
  model.add(Dense(
        # tune number of units
        units = hp.Int('units', min_value=32, max_value=512, step=32),
        # tune the activation function to use
        activation=hp.Choice("activation", ["relu", "tanh"])))
  model.add(Dense(10, activation="softmax"))
  
  learning_rate = hp.Float("lr", min_value=1e-4, max_value=1e-2, sampling="log")
  model.compile(optimizer=keras.optimizers.Adam(learning_rate=learning_rate), 
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), 
              metrics=['accuracy'])
  
  return model

Build and Compile the model

In [ ]:
build_model(keras_tuner.HyperParameters())

Create the tuner

In [ ]:
tuner = keras_tuner.RandomSearch(
    hypermodel=build_model,
    objective='val_accuracy',
    max_trials=100,
    executions_per_trial=2,
    directory='my_dir',
    project_name = "cifar10-image-classification-cnn")

Execute a search for the best model

In [ ]:
stop_early =  callback_95()

tuner.search(X_train, y_train, epochs=100, validation_data=(X_val, y_val), callbacks=[stop_early])
best_model = tuner.get_best_models()[0]
print(f"Search Space Summary:\n{tuner.search_space_summary}")